In [1]:
# Import and Install Dependencies
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
# Keypoints using MP Holistic
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                    # Image is no longer writeable
    results = model.process(image)                   # Make prediction
    image.flags.writeable = True                     # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [9]:
# Extract Keypoint Values
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # Pose (132) + LH (63) + RH (63) = 258 Keypoints. இது மாடலின் உள்ளீட்டிற்குச் சரியாகப் பொருந்துகிறது.
    return np.concatenate([pose, lh, rh])

In [11]:
# Paths 
DATA_PATH = r"C:\Users\HP\Desktop\RP\RP\Models Training\Jeran\VIDEO_DATA"
VIDEO_PATH = r"C:\Users\HP\Desktop\RP\RP\Models Training\Jeran\Video_Input" 

# Variables
# ⚠️ திருத்தம் 1: 'Thank you'-வை நீக்கி, உங்கள் பயிற்சி குறியீட்டில் உள்ள 20 Actions-ஐ உறுதிப்படுத்தவும்
actions = np.array([
    'Beautiful', 'Black', 'Drink', 'Eat', 'Five',
    'Good_morning', 'Good_night', 'Hello',
    'Love', 'Mother', 'One', 'Run', 'Sleep','Ten',
    'Three', 'Today', 'Tomorrow', 'Two', 'Yesterday'
])   
no_videos = 50                  
sequence_length = 31# Frames per vid

In [13]:
# Setup Folders for Collection (This creates folders like VIDEO_DATA/A/0)
for action in actions: 
    for video in range(no_videos):
        folder_path = os.path.join(DATA_PATH, action, str(video))
        os.makedirs(folder_path, exist_ok=True)

In [15]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through videos
        for video in range(no_videos):
            # Set path to the current video file
            video_path = os.path.join(VIDEO_PATH, action, str(video) + ".mp4")
            
            # கோப்பு இல்லையெனில் ஸ்கிப் செய்யவும்
            if not os.path.exists(video_path):
                print(f"Skipping: {video_path} not found.")
                continue

            cap = cv2.VideoCapture(video_path)
            
            # ⚠️ திருத்தம் 2: Frame Selection Logic மாற்றப்பட்டுள்ளது
            frame_num = 0
            
            # Loop through video frames
            while cap.isOpened():
                # Read frame from the video
                ret, frame = cap.read()
                
                # Check if video has ended OR if we have collected 45 frames
                if not ret or frame_num >= sequence_length:
                    break

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                # Draw landmarks
                draw_styled_landmarks(image, results)

                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, video), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500) # 0.5 விநாடி காத்திருக்கவும்
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, video), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(video), str(frame_num))
                np.save(npy_path, keypoints)
                
                frame_num += 1 # Frame எண்ணை அதிகரிக்கவும்

                # Break gracefully
                if cv2.waitKey(1) & 0xFF == ord('q'): # சிறிய தாமதம்
                    break
                    
            cap.release()
            # cv2.destroyAllWindows() # லூபிற்கு வெளியே கடைசியாக அழைப்போம்

    cv2.destroyAllWindows() # அனைத்து லூப்களும் முடிந்த பிறகு மூடவும்
    print("Data Collection Complete.")

Data Collection Complete.
